In [1]:
from pathlib import Path

import pdfplumber
import pandas as pd
import json

In [25]:
!pip uninstall camelot

^C


In [28]:
input_file_path = Path.cwd().parent / "input data"

# Get tables

In [4]:
def detect_table (page, table_setting_list): 
    best_table_settings = None
    best_table_len = float('-inf')
    
    for ts in table_settings_list:
        tables = page.extract_tables(table_settings=ts)
        tables_len = []
        for table in tables:
            tables_len.append(len(table))

        if max(tables_len) > best_table_len:
            best_table_len = max(tables_len)
            best_table = table
            best_table_setting = ts
            
    return best_table_setting #best table setting for a page

In [5]:
def get_table(page,best_table_setting): #for long-format tables

    tables = page.extract_tables(table_settings=best_table_setting)
    
    tables_str = [] 
    
    for table in tables:
        headers =  table[0]
        rows = table[1:]
        
        table_str = {} 
        for row in rows:
            table_str[row[0]] = {}
            for i in range(1,len(row)):
                table_str[row[0]][headers[i]] = row[i]
        tables_str.append(table_str)
        
    return tables_str

In [8]:
table_settings_list = [
    {"vertical_strategy": "text", "horizontal_strategy": "text", "snap_tolerance": 10, "intersection_tolerance": 150},
    {"vertical_strategy": "text", "horizontal_strategy": "text", "snap_tolerance": 10, "intersection_tolerance": 30},
    {"vertical_strategy": "lines", "horizontal_strategy": "lines", "snap_tolerance": 10, "intersection_tolerance": 30},

]

with pdfplumber.open(input_file_path / "pdfwithtable_easy.pdf") as pdf:
    pages = pdf.pages
    
    for idx, page in enumerate(pages[0:1],1):
        im = page.to_image()
        best_setting = detect_table(page,table_settings_list)
        im.reset().debug_tablefinder(best_setting).show()
        tables = get_table(page,best_setting)
        print("Best table settings: ", best_setting)
        print("List of tables in page: ",idx )
        print(tables)

Best table settings:  {'vertical_strategy': 'lines', 'horizontal_strategy': 'lines', 'snap_tolerance': 10, 'intersection_tolerance': 30}
List of tables in page:  1
[{'Duong Dinh R\nSenior Manager\nduong.dinh@18v cMsca.rccohm 2.v0n1 1\n+84 28 3914 3588 ext. 140 V\nstr\nYo\nw\nfol\nT\n20\nPr\nB\nw': {}}, {None: {None: None, 'gure 1: W\nState\nO’ship': 'gure 2: W\nEPS g\n2023F', 'ater com\nForeign\nLimit %': 'ater com\nEPS g\n2024F', 'panies –\nForeign\nAvail': 'panies –\nP/E TTM\n(x)', 'Key data\nADTV\n30D': 'Summar\nP/E\n2022 (x)'}, 'BWE\nTDM': {None: '63.3%\n12.9%', 'gure 1: W\nState\nO’ship': '%\n14.7%\n39.3%', 'ater com\nForeign\nLimit %': '%\n50.3%\n25.9%', 'panies –\nForeign\nAvail': '13.4x\n18.0x', 'Key data\nADTV\n30D': '13.4x\n18.0x'}}]


# Get text

In [35]:
with pdfplumber.open(input_file_path / "pdfwithtable_easy.pdf") as pdf:
    pages = pdf.pages[0:2]
    for page in pages:
        print(page.extract_text())

Water Sector Sector Update
Duong Dinh Residential segment to support volume growth
Senior Manager
duong.dinh@18v cMsca.rccohm 2.v0n1 1
+84 28 3914 3588 ext. 140 Volume continues to grow, supporting BWE and TDM’s 2023 profit. This growth will be led by
strong residential water demand. We lower our projected 2023 volume growth for BWE from 12%
YoY to 7% YoY, which is driven by 9% YoY residential water volume growth and 5% YoY industrial
water volume growth. For TDM, we raise our projected 2023 volume to 79 million cbm (+13% YoY)
following its 2022 results that beat our expectations. We attribute the stronger volume growth for
TDM vs BWE in 2022-2023 to 1) TDM being impacted more severely compared to BWE in 2020-
2021, and 2) TDM’s major water plants being in the most populated industrial hub of Binh Duong
Province (Di An District), which faces less risk of an economic slowdown compared to other districts.
BWE to acquire a 20%-100% stake in four water companies in Long An Province and one